A particle of mass m moves along a street, a curve in $\R^2$\
A force F acts on the particle, in the direction of the tangent at the street at the point where the particle is.\
Gravity points in the negative Y direction.\
A speed dependent friction also may act in the same direction.\
The goal is to move the particle from ist rest location, x(0) = 0  to its final location x(duration) = $x_1$,  with the objective: $\int_0^{\text{duration}} F(k)^2 \, dk$ = minimal.

In [ ]:
import sympy.physics.mechanics as me
from collections import OrderedDict
import time
import numpy as np
import sympy as sm
from opty.direct_collocation import Problem
#from opty.utils import building_docs
import matplotlib.pyplot as plt
import matplotlib
#import matplotlib.animation as animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.animation import FuncAnimation

Defines the shape of the street (strasse = German for street)\
One is a single hump function, the other one a double hump function.

In [ ]:
def strasse(x, a, b):
    return a * x**2 * sm.exp((b - x))  #a * sm.exp(x**2 * (1 - x**2)) # 

Set up **Kane's equations of motion**\
The control force F (and a speed dependent friction, if so desired) act in the direction of the tanget to the street at the point where the particle is at that moment.\
For the angle of the tangent with the X axis this holds: $\tan(\alpha) = \frac{d}{dx} strasse(x, a, b) \longrightarrow \alpha = \tan^{-1}(\frac{d}{dx} strasse(x, a, b))$\
So, the force in the N.x direction is $F_x = F\cdot \cos(\alpha)$, in the N.y direction it is $F_x = F\cdot \cos(\alpha)$

In [ ]:
start = time.time()

import sympy.physics.mechanics as me
import sympy as sm

N = me.ReferenceFrame('N')                                                                                           
O = me.Point('O')                                                                                                         
O.set_vel(N, 0)                                                                                                        
t = me.dynamicsymbols._t


P0 = me.Point('P0')                                                             
x = me.dynamicsymbols('x')    
ux = me.dynamicsymbols('u_x')                                                                                                                                                    
F = me.dynamicsymbols('F')                                        # Force applied to the particle                                                  

m, g, reibung = sm.symbols('m, g, reibung')     
a, b = sm.symbols('a b')                                                                         


P0.set_pos(O, x * N.x + strasse(x, a, b) * N.y)
P0.set_vel(N, ux * N.x + strasse(x, a, b).diff(x)*ux * N.y)
BODY = [me.Particle('P0', P0, m)]

# The control force and the friction are acting in the direction of the tangent at the street at the point whre the particle is.
alpha = sm.atan(strasse(x, a, b).diff(x))
FL = [(P0, -m*g*N.y + F*(sm.cos(alpha)*N.x + sm.sin(alpha)*N.y) - reibung*ux*(sm.cos(alpha)*N.x + sm.sin(alpha)*N.y))]     

kd = sm.Matrix([ux - x.diff(t)])      

q_ind = [x]
u_ind = [ux]
 
KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)
EOM = kd.col_join(fr + frstar) 
print('EOM DS', me.find_dynamicsymbols(EOM))
print('EOM FS', EOM.free_symbols)
EOM.simplify()
EOM

Set up the machinery for **opty**

In [ ]:
state_symbols = tuple((x, ux))
laenge = len(state_symbols)
constant_symbols = (m, g, reibung, a, b)
specified_symbols = (F,)

duration  = 7.                                              # The duration of the simulation.
num_nodes =  1000                                           # The number of nodes to use in the direct collocation problem.

interval_value = duration / (num_nodes - 1)

# Specify the known system parameters.
par_map = OrderedDict()
par_map[m]       = 1.0                                      # Mass of the block
par_map[g]       = 9.81                                     # gravity
par_map[reibung] = 0.                                       # Friction coefficient between the block and the ground
par_map[a]       = 1.                                       # Parameter of the street
par_map[b]       = 2.                                       # Parameter of the street   

# Specify the objective function and it's gradient.
def obj(free): 
    """Minimize the sum of the squares of the control torques."""
    Fx = free[laenge * num_nodes: (laenge + 1) * num_nodes] 
    return interval_value * np.sum(Fx**2)

def obj_grad(free):
    grad = np.zeros_like(free)
    grad[laenge * num_nodes: (laenge + 1) * num_nodes]       = 2.0 * interval_value * free[laenge * num_nodes: (laenge + 1) * num_nodes]
    return grad

Create an **optimization problem** and solve it\
A solution is found easily, but one has to play around with the bounds a little bit.\
For example, unless I set {ux: (0, 1000)} the block may move to the crest of the street, stay there, moveback to its starting point and the on to its final point. Obviously not optimal.

In [ ]:
# Create an optimization problem.
# This portion is copied from Timo's code
t0, tf = 0.0, duration                    # The initial and final times

#======================================================================================================
methode = 'backward euler'  # The integration method to use. backward euler or midpoint are the choices
#======================================================================================================

initial_guess = np.ones((len(state_symbols) + len(specified_symbols)) * num_nodes) * 0.01

# Timo's comments:
# Opty want instance constraints, basically things like `q2(2.0) - 3.141` to specify that q2 should be 3.141 when t=2.0
# I personally like to rewrite it to initial state and final state constraints and then translate them into the set of equations opty wants.    
initial_state_constraints = {
                            x: 0.,
                            ux: 0.
                            }


final_state_constraints    = {
                             x: 10.,
                             ux: 0.,
                             }    
    
instance_constraints = tuple(xi.subs({t: t0}) - xi_val for xi, xi_val in initial_state_constraints.items()) + tuple(xi.subs({t: tf}) - xi_val for xi, xi_val in final_state_constraints.items())
print('initial constraints:', instance_constraints)
bounds = {F: (-20., 20.), x: (initial_state_constraints[x], final_state_constraints[x]), ux: (0., 1000.)}
               
prob = Problem(obj, obj_grad, EOM, state_symbols, num_nodes, interval_value,
        known_parameter_map=par_map,
        instance_constraints=instance_constraints,
        bounds=bounds,
        integration_method=methode)

prob.add_option('max_iter', 3000)           # default is 3000
# Find the optimal solution.
solution, info = prob.solve(initial_guess)
print('message from optimizer:', info['status_msg'])
print(f'objective value {obj(solution):,.1f} \n')
prob.plot_objective_value()

Plot location and speed of the particle, and also the force acting on it.

In [ ]:
anzahl = len(state_symbols) + len(specified_symbols)
fig, ax = plt.subplots(anzahl, 1, figsize=(10, 3*anzahl), sharex=True)
times = np.linspace(0.0, duration, num=num_nodes)

for i, j in enumerate(state_symbols + specified_symbols):
    ax[i].plot(times, solution[i * num_nodes:(i + 1) * num_nodes])
    ax[i].grid()
    ax[i].set_ylabel(str(j))
ax[-1].set_xlabel('time')
ax[0]. set_title('Generalized coordinates and speeds')
ax[-1]. set_title('Control force');

aminate the contraption

In [ ]:
strasse1 = strasse(x, a, b)
strasse_lam = sm.lambdify((x, a, b), strasse1, cse=True)    

P0_x = solution[:num_nodes]
P0_y = strasse_lam(P0_x, par_map[a], par_map[b])


# needed to give the picture the right size.
xmin = np.min(P0_x)
xmax = np.max(P0_x)
ymin = np.min(P0_y)
ymax = np.max(P0_y)


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.set_xlim(xmin-1, xmax + 1.)
ax.set_ylim(ymin-1, ymax + 1.)

ax.grid()
strasse_x = np.linspace(xmin, xmax, 100)
ax.plot(strasse_x, strasse_lam(strasse_x, par_map[a], par_map[b]), color='black', linestyle='-', linewidth=1)   # the ground
ax.axvline(initial_state_constraints[x], color='r', linestyle='--', linewidth=1)                                # the initial position
ax.axvline(final_state_constraints[x], color='green', linestyle='--', linewidth=1)                              # the final position

# Initialize the block
line1, = ax.plot([], [], color='blue', marker='o', markersize=12)                                               # the sliding block

# Function to update the plot for each animation frame
def update(frame):
    message = (f'running time {times[frame]:.2f} sec \n the red line is the initial position, the green line is the final position')
    ax.set_title(message, fontsize=15)
    
    line1.set_data([P0_x[frame]], [P0_y[frame]])

    return line1,

# Set labels and legend
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

# Create the animation
animation = FuncAnimation(fig, update, frames=range(len(P0_x)), interval=2000*np.max(times) / num_nodes, blit=True)
plt.close(fig)  # Prevents the final image from being displayed directly below the animation
# Show the plot
display(HTML(animation.to_jshtml()))
print(f'it took {time.time()-start:.1f} seconds to run the code.')